In [1]:
import numpy as np
from scipy.stats import f, t

In [2]:
def check_H_0(p_value, alpha):
    if p_value <= alpha:
        print('Reject H_0')
    else:
        print('Accept H_0')

In [3]:
ALPHA = 0.05

DATA_SOURCE_PATH_5_18 = 'data_samples/T5_6_PILOT.DAT'
DATA_SOURCE_PATH_5_20 = 'data_samples/T5_8_GOODS.DAT'

$T^{2} = \dfrac{n_{1}n_{2}}{n_{1}+n_{2}}(\bar{y_{1}} − \bar{y_{2}})^{'} S^{-1}_{pl}(\bar{y_{1}} − \bar{y_{2}})$

In [4]:
def calc_hotteling(mean1, mean2, S, n1, n2, p):
    T_2 = (mean1.T - mean2.T) @ np.linalg.pinv(S) @ (mean1 - mean2) * ((n1 * n2) /(n1 + n2))
    
    statistic = T_2 * (n1 + n2 - p - 1) / (p * (n1 + n2 - 2))
    F = f(p, n1 + n2 - p - 1)
    p_value = 1 - F.cdf(statistic)
    return T_2, statistic, p_value

## 5.18

Twenty engineer apprentices and 20 pilots were given six tests (Travers 1939). The variables were:

+ $y_1$ = intelligence
+ $y_2$ = form relations
+ $y_3$ = dynamometer
+ $y_4$ = dotting
+ $y_5$ = sensory motor coordination
+ $y_6$ = perseveration

In [5]:
def load_and_prepare_data_5_18(pilot_data_path):
    data = np.loadtxt(pilot_data_path, dtype=int)

    pilots_data = data[:20, 1:]
    nx = pilots_data.shape[0]
    p = pilots_data.shape[1]

    engineers_data = data[20:, 1:]
    ny = engineers_data.shape[0]

    engineers_mean = np.mean(engineers_data, axis=0)
    pilots_mean = np.mean(pilots_data, axis=0)

    engineers_cov = np.cov(engineers_data, rowvar=False)
    pilots_cov = np.cov(pilots_data, rowvar=False)

    S = ((nx - 1) * engineers_cov + (ny - 1) * pilots_cov) / (nx + ny - 2)
    
    return pilots_data, nx, p, engineers_data, ny, engineers_mean, pilots_mean, engineers_cov, pilots_cov, S


In [6]:
pilots_data, nx, p, engineers_data, ny, engineers_mean, pilots_mean, engineers_cov, pilots_cov, S = load_and_prepare_data_5_18(DATA_SOURCE_PATH_5_18)

a) Test $H_{0}:\mu_{1}=\mu_{2}$.

In [7]:
T_2, statistic, p_value = calc_hotteling(engineers_mean, pilots_mean, S, nx, ny, p)
T_2, statistic, p_value

(66.6604389902222, 9.648221432795319, 3.851202317717295e-06)

In [8]:
check_H_0(p_value, ALPHA)

Reject H_0


$p_{val} \leq 0.05$ -> reject $H_0$ 

b) If the $T^2-test$ in part (a) rejects $H_0$, carry out a _t-test_ for each variable, as in (5.15)

$t_{j} = \frac{\bar{y_{1j}} − \bar{y_{2j}}}{\sqrt{[(n_{1}+n_{2})/n_{1}n_{2}]s_{jj}}} $ , $j = 1,...6$

In [9]:
df = ny + nx - 2
cv = t.ppf(1.0 - ALPHA, df)

t_stats = np.asarray([
    (engineers_mean[j] - pilots_mean[j]) / np.sqrt(((nx + ny) / (nx * ny)) * S[j,j])
    for j in range(p)
])

p_vals = np.asarray([
    (1 - t.cdf(abs(t_stats[j]), df)) * 2
    for j in range(p)
])

p_vals > ALPHA

array([ True, False, False, False,  True, False])

Accept $H_0$ only for $\mu_1$ and $\mu_5$

## 5.20

Various aspects of economic cycles were measured for consumers’ goods and producers’ goods by Tintner (1946). The variables are:

+ $y_1$ = length of cycle
+ $y_2$ = percentage of rising prices
+ $y_3$ = cyclical amplitude
+ $y_4$ = rate of change

In [10]:
def load_and_prepare_data_5_20(data_file):
    data = np.loadtxt(data_file, dtype=float)

    cons_data = data[:9, 1:]
    nx = cons_data.shape[0]
    p = cons_data.shape[1]

    prod_data = data[9:, 1:]
    ny = prod_data.shape[0]

    cons_mean = np.mean(cons_data, axis=0)
    prod_mean = np.mean(prod_data, axis=0)

    cons_cov = np.cov(cons_data, rowvar=False)
    prod_cov = np.cov(prod_data, rowvar=False)

    S = ((nx - 1) * cons_cov + (ny - 1) * prod_cov) / (nx + ny - 2)

    return cons_data, nx, p, prod_data, ny, cons_mean, prod_mean, cons_cov, prod_cov, S


In [11]:
cons_data, nx, p, prod_data, ny, cons_mean, prod_mean, cons_cov, prod_cov, S = load_and_prepare_data_5_20(DATA_SOURCE_PATH_5_20)

a) Test $H_{0}:\mu_{1}=\mu_{2}$ using $T^2$.

In [12]:
T_2, statistic, p_value = calc_hotteling(cons_mean, prod_mean, S, nx, ny, p)
T_2, statistic, p_value

(18.46248029300786, 2.823673456577673, 0.061072768410669886)

In [13]:
check_H_0(p_value, ALPHA)

Accept H_0
